In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import glob
import os
import pandas as pd
from turbo_seti.find_event.find_event_pipeline import find_event_pipeline
from turbo_seti.find_event.plot_event_pipeline import plot_event_pipeline

In [5]:
# outdir = '/datax/scratch/benjb/bl_nearby_stars/seticore_output/within_5.1_pc/'

# df = pd.read_csv('/datax/scratch/benjb/bl_nearby_stars/unique_star_cadences_catalog_within_5.1_pc_no_band.csv')
# h5list = df['.h5 path'].values

# dfr = pd.read_csv('/datax/scratch/benjb/bl_nearby_stars/star_cadences_catalog_with_receiver.csv')

# rcvrlist = dfr['Receiver'].values

# rcvrvec = []

# for h5 in h5list:
#     rcvrs = rcvrlist[np.where(dfr['.h5 path'].values == h5)[0]]
#     rcvr = np.unique(rcvrs)[0]
#     rcvrvec.append(rcvr)

# rcvrvec = np.array(rcvrvec)

# df2 = pd.DataFrame({'Receiver': rcvrvec})
# dfn = df.join(df2)

# datlist = []

# for h5 in h5list:
#     basename = os.path.basename(h5)
#     dat = outdir + basename[:-2] + 'dat'
#     datlist.append(dat)

# dff = df.join(pd.DataFrame({'.dat path': datlist})).join(df2)

# dff.to_csv('/datax/scratch/benjb/bl_nearby_stars/star_cadences_catalog.csv')

In [2]:
df = pd.read_csv('/datax/scratch/benjb/bl_nearby_stars/star_cadences_5.1_catalog.csv')

In [3]:
print(np.unique(df['Receiver'].values))

['Rcvr1_2' 'Rcvr2_3' 'Rcvr4_6' 'Rcvr8_10']


In [7]:
h5_path = '/datax/scratch/benjb/bl_nearby_stars/seticore_output/within_5.1_pc/blc13_guppi_60331_80780_GJ1002_0129.rawspec.0000.dat'
print(df.iloc[np.where(df['.dat path'].values == h5_path)[0]]['.h5 path'].values)

['/datag/pipeline/AGBT23B_999_31/blc13_blp13/blc13_guppi_60331_80780_GJ1002_0129.rawspec.0000.h5']


In [ ]:
#for i, dat_path in enumerate(df['.dat path'].values):
#    s = dat_path.split('blc')
#    if ('spliced' in dat_path) & (s[1][2] == '_'):
#        print(df['.h5 path'].values[i])
    #if not os.path.exists(dat_path):
    #    print(dat_path)

/datag/pipeline/AGBT16A_999_227/collate/spliced_blc04_guppi_57563_75522_GJ412A_0027.gpuspec.0000.h5
/datag/pipeline/AGBT16A_999_227/collate/spliced_blc04_guppi_57563_75889_HIP53051_0028.gpuspec.0000.h5
/datag/pipeline/AGBT16A_999_227/collate/spliced_blc04_guppi_57563_76259_GJ412A_0029.gpuspec.0000.h5
/datag/pipeline/AGBT16A_999_227/collate/spliced_blc04_guppi_57563_76624_HIP53160_0030.gpuspec.0000.h5
/datag/pipeline/AGBT16A_999_227/collate/spliced_blc04_guppi_57563_76991_GJ412A_0031.gpuspec.0000.h5
/datag/pipeline/AGBT16A_999_227/collate/spliced_blc04_guppi_57563_77362_HIP53283_0032.gpuspec.0000.h5
/datag/pipeline/AGBT16A_999_227/collate/spliced_blc04_guppi_57563_77736_GJ412B_0033.gpuspec.0000.h5
/datag/pipeline/AGBT16A_999_227/collate/spliced_blc04_guppi_57563_78122_HIP53051_0034.gpuspec.0000.h5
/datag/pipeline/AGBT16A_999_227/collate/spliced_blc04_guppi_57563_78507_GJ412B_0035.gpuspec.0000.h5
/datag/pipeline/AGBT16A_999_227/collate/spliced_blc04_guppi_57563_78888_HIP53160_0036.gpuspe

In [5]:
full_target_list = df['Target'].values
full_cadence_id_list = df['Cadence ID'].values
full_h5_list = df['.h5 path'].values
full_dat_list = df['.dat path'].values
full_receiver_list = df['Receiver'].values

data_loc = '/datax/scratch/benjb/bl_nearby_stars/seticore_output/within_5.1_pc'

start_counter = 0

for i in range(len(full_h5_list)//6):

    # for each set of 6 files in the 5.1 pc sample, run FindEvent and PlotEvent

    h5_list = full_h5_list[6*i:6*i+6]
    dat_list = full_dat_list[6*i:6*i+6]
    target = full_target_list[6*i]
    cadence_id = full_cadence_id_list[6*i]
    receiver = full_receiver_list[6*i]
    s = dat_list[0].split('blc')
    if ('spliced' in dat_list[0]) & (s[1][2] == '_'):
        node = 'blc'+s[1][:2]
    else:
        node = os.path.basename(full_h5_list[6*i])[:5]

    if not cadence_id == 4051:
        continue

    print(node)
    print(dat_list[0])

    # if (target == 'GJ412B') & (node == 'blc04') & (receiver == 'Rcvr1_2'):
    #     start_counter += 1
    
    # if start_counter == 0:
    #     continue

    h5_list_path = os.path.join(data_loc, f'{cadence_id}_{target}_{receiver}_{node}_h5_files.lst')
    with open(h5_list_path, 'w') as f:
        for h5_path in h5_list:
            f.write(h5_path + '\n')

    dat_list_path = os.path.join(data_loc, f'{cadence_id}_{target}_{receiver}_{node}_dat_files.lst')
    with open(dat_list_path, 'w') as f:
        for dat_path in dat_list:
            f.write(dat_path + '\n')

    csvf_path = f'{data_loc}/{cadence_id}_{target}_{receiver}_{node}_found_event_table_3.csv'

    find_event_pipeline(dat_list_path, 
                    h5_list_path,
                    #SNR_cut=2,
                    check_zero_drift = True,
                    filter_threshold = 3, 
                    number_in_cadence = len(dat_list),
                    csv_name=csvf_path, 
                    saving=True
                    )
    
    if not os.path.exists(csvf_path):
        print('No event CSV found. Continuing ...')
        continue

    # trimming regions of insensitivity
    event_df = pd.read_csv(csvf_path)
    if receiver == 'Rcvr1_2':
        trimmed_event_df = event_df.iloc[
            np.where((event_df['Freq'].values > 1100) & (event_df['Freq'].values < 1200) |
                    (event_df['Freq'].values > 1340) & (event_df['Freq'].values < 1900))[0]
            ]
    elif receiver == 'Rcvr2_3':
        trimmed_event_df = event_df.iloc[
            np.where((event_df['Freq'].values > 1800) & (event_df['Freq'].values < 2300) |
                    (event_df['Freq'].values > 2360) & (event_df['Freq'].values < 2700))[0]
            ]
    elif receiver == 'Rcvr4_6':
        trimmed_event_df = event_df.iloc[
            np.where((event_df['Freq'].values > 4000) & (event_df['Freq'].values < 7800))[0]
            ]
    elif receiver == 'Rcvr8_10':
        trimmed_event_df = event_df.iloc[
            np.where((event_df['Freq'].values > 7800) & (event_df['Freq'].values < 11200))[0]
            ]
    
    trimmed_event_df.dropna(subset=['in_n_ons'], inplace=True)

    trimmed_event_df.to_csv(f'{data_loc}/{cadence_id}_{target}_{receiver}_{node}_f3_trimmed.csv')
    tcsvf_path = f'{data_loc}/{cadence_id}_{target}_{receiver}_{node}_f3_trimmed.csv'
    
    if len(trimmed_event_df) > 0:
        plot_event_pipeline(tcsvf_path, # full path of the CSV file built by find_event_pipeline()
                h5_list_path, # full path of text file containing the list of .h5 files
                filter_spec='f{}'.format(3), # filter threshold
                user_validation=False, # Non-interactive
                plot_dir=data_loc+'/pngs/') 

splic
/datax/scratch/benjb/bl_nearby_stars/seticore_output/within_5.1_pc/spliced_blc00010203050607_guppi_57563_75522_GJ412A_0027.gpuspec.0000.dat

===========   BEGINNING FIND_EVENT PIPELINE   ===========

Assuming the first observation is an ON
find_event_pipeline INFO     file=spliced_blc00010203050607_guppi_57563_75522_GJ412A_0027.gpuspec.0000.dat, tstart=57563.87409722222, source_name=GJ412A, fch1=2251.464842353016, foff=-2.7939677238464355e-06, nchans=469762048
find_event_pipeline INFO     file=spliced_blc00010203050607_guppi_57563_75889_HIP53051_0028.gpuspec.0000.dat, tstart=57563.87834490741, source_name=HIP53051, fch1=2251.464842353016, foff=-2.7939677238464355e-06, nchans=469762048
find_event_pipeline INFO     file=spliced_blc00010203050607_guppi_57563_76259_GJ412A_0029.gpuspec.0000.dat, tstart=57563.882627314815, source_name=GJ412A, fch1=2251.464842353016, foff=-2.7939677238464355e-06, nchans=469762048
find_event_pipeline INFO     file=spliced_blc00010203050607_guppi_57563_76

In [1]:
print('/datag/pipeline/AGBT16A_999_227/collate/spliced_blc04_guppi_57563_77736_GJ412B_0033.gpuspec.0000.h5')

/datag/pipeline/AGBT16A_999_227/collate/spliced_blc04_guppi_57563_77736_GJ412B_0033.gpuspec.0000.h5


In [2]:
print('/datax/scratch/benjb/bl_nearby_stars/seticore_output/within_5.1_pc/spliced_blc04_guppi_57563_77736_GJ412B_0033.gpuspec.0000.dat')

/datax/scratch/benjb/bl_nearby_stars/seticore_output/within_5.1_pc/spliced_blc04_guppi_57563_77736_GJ412B_0033.gpuspec.0000.dat


In [3]:
print('/datax/scratch/benjb/bl_nearby_stars/seticore_output/within_5.1_pc/4057_GJ412B_Rcvr1_2_splic_f3_trimmed.csv')

/datax/scratch/benjb/bl_nearby_stars/seticore_output/within_5.1_pc/4057_GJ412B_Rcvr1_2_splic_f3_trimmed.csv


In [5]:
print('/datax/scratch/benjb/bl_nearby_stars/seticore_output/within_5.1_pc/spliced_blc00010203050607_guppi_57563_77736_GJ412B_0033.gpuspec.0000.dat')

/datax/scratch/benjb/bl_nearby_stars/seticore_output/within_5.1_pc/spliced_blc00010203050607_guppi_57563_77736_GJ412B_0033.gpuspec.0000.dat
